# MPI

MPI (Message Passing Interface) is a standard for writing parallel programs that can run on a distributed memory system. It is widely used in the HPC (High Performance Computing) community. There are implementations of MPI for many programming languages, including C, C++, Fortran, and Python. Much of the functionality and many of the commands we will see here will be similar in other languages. 

There are multiple implementations of MPI, including [OpenMPI](https://www.open-mpi.org/), [MPICH](https://www.mpich.org/), and [Intel MPI](https://www.intel.com/content/www/us/en/developer/tools/oneapi/mpi-library.html). If you are running this notebook in a Github Codespace, MPICH will already be installed. If you want to install MPICH on your local system you can follow the instructions [here](https://www.mpich.org/downloads/).

We will also be using the `mpi4py` package which is a Python wrapper for MPI. This is already installed if you'r running this notebook in a GitHUb Codespace, or you can install it locally using pip:

```bash
pip install mpi4py
```

Unlike the other methods we have seen so far, we run MPI from the terminal using the `mpiexec` command. We can run a Python script named `python_script.py` using MPI like this:

```bash
mpiexec -n 4 python python_script.py
```
In this command, the `-n 4` flag tells MPI to run the script using 4 processes. The text `python python_script.py` tells which command MPI should be running on each process. This will run the script 4 times in parallel.

## Ranks

Each copy of the code will be executed on its own process and will be identified by its "rank". The rank is a unique integer identifier for each process which can be accessed using the `mpi4py.MPI.COMM_WORLD.Get_rank()` function. We can also get the number of ranks using the method `Get_rank()`. The code below shows how to access the rank:

```python
# Run this script with the terminal command `mpiexec -n 4 python get_rank.py`

import mpi4py.MPI as MPI

# Get a reference to the current MPI.COMM_WORLD communicator
comm = MPI.COMM_WORLD

# Get the total number of ranks in the communicator
n_rank = comm.Get_size()

# Get the rank of the current process
rank = comm.Get_rank()

# Print the rank of the current process
print(f'This script is being run by Rank {rank} out of {n_rank} total ranks')
```

This code can be found in the file `04_mpi_scripts/get_rank.py`. To run this command we will need to change directory in the terminal using the command:

```bash
cd 04_mpi_scripts
```

and run the script using MPI using the command:

```bash
mpiexec -n 4 python 04_mpi_scripts/get_rank.py
```

## Communicating Between Ranks

In the above example, we access the variable `MPI.COMM_WORLD` which references a communicator. We can use a communicator to send messages between the processes it contains. The communicator `MPI.COMM_WORLD` contains all the processes that are running the script. It is possible to create other communicators which contain only a subset of the processes, which can be useful for more complex parallel programs, but we won't be covering that here.

We can send messages between ranks using the `send` and `recv` methods of the communicator. The code below shows how to send a message from rank 0 to rank 1:

```python
from mpi4py import MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # If we're in rank 0, send a dictionary to rank 1
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1)
elif rank == 1:
    # If we're in rank 1, receive the dictionary from rank 0
    data = comm.recv(source=0)
    print(rank, data)
```

In the example above the `send` method is used to send a dictionary from rank 0 to rank 1. By including it in the if-block, we make sure it is only called by the rank 0. The first argument to `send` is the data to be sent. We also specify the destination rank so the message can be sent to rank 1. The `recv` method is called from rank 1 to receive the message from rank 0. The `source` argument specifies the rank of the process that sent the message. The data which is received is saved into the variable `data` and then printed. As we can see, we can send any type of data between ranks, including dictionaries, lists, and numpy arrays.

Both `send` and `recv` block, meaning that the program will wait at the `send` line until the message has been received by the destination rank, and will wait at the `recv` line until the message has been sent by the source rank. This means we need to plan carefully to make sure that the program doesn't get deadlocked. For example, the code below will deadlock as both ranks are waiting for the other to receive a message:

```python
from mpi4py import MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # Send a message to rank 1
    comm.send("Hello from rank 0", dest=1)
    # Receive the message from rank 1
    data = comm.recv(source=1)
elif rank == 1:
    # Send a message to rank 0
    comm.send("Hello from rank 1", dest=0)
    # Receive the message from rank 0
    data = comm.recv(source=0)
```

This code can be found in the file [`04_mpi_scripts/deadlock.py`](04_mpi_scripts/deadlock.py) and should be run with two processes.

## Non-Blocking Communication

To avoid deadlocks, we can use non-blocking communication. This allows the program to continue running while the message is being sent or received. This can be done using the `isend` and `irecv` functions. These returned a `Request` object. We can use the `wait` method of the `Request` object to wait until the message has been sent or received. If the `Request` object was created by `irecv` the `wait` method waits until it receives a value, then returns the value received. Using non-blocking communication can free up processes to do other work while the communication is pending. The code below shows how to use non-blocking communication:

```python
from mpi4py import MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # Send the integer 100 to rank 1
    req = comm.isend(100,dest=1)
    # Wait for the request to complete
    req.wait()
elif rank == 1:
    # Receive the integer from rank 0
    req = comm.irecv(source=0)
    # Wait for the request to complete and get the data
    data = req.wait()
    print(data)
```

This code can be found in the file [`04_mpi_scripts/non_blocking.py`](04_mpi_scripts/non_blocking.py) and should be run with two processes.

## Sending Numpy Arrays

The methods we've seen before like `send`, `recv`, `isend`, and `irecv` send Python objects between ranks using a process called [pickling](https://docs.python.org/3/library/pickle.html). This process allows an arbitrarily complex object to be serialized so they can be sent between ranks. Whilst flexible in terms of the types of object that can be sent, the process of pickling and unpicking adds a significant performance overhead to the sending of data between ranks.

However, some data types in Python, such as Numpy arrays, do not need to be pickled to be sent between ranks, which can speed up communication significantly. This can be done using functions with similar names to those we have already seen, except they begin with a capital letter, such as `Send` and `Recv`.

The syntax we have to use is a little different than before. We need to prepare an object to receive the data before we call the `Recv` method. This array should be a Numpy array with the same shape and data type as the array we are sending. This prepares a section of the memory in receiving rank to receive the data and is known as a buffer. The function `numpy.empty` is an efficient way to create this buffer. It will allocate the memory for th Numpy array but will not initialize it, meaning it will contain junk values. This is faster than using `numpy.zeros` which would initialize the array to zeros. For many Numpy functions, including `empty`, the `dtype` argument is used to [specify the data type](https://numpy.org/doc/2.1/reference/arrays.dtypes.html) of the array. There are a few ways to do this, but one ay is to use the Python type names such as `int`, `float`, `complex`, etc.

The code below shows how to send a Numpy array between ranks:

```python
from mpi4py import MPI
import numpy as np

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # If we're in rank 0, create an array of ten integers to send
    data = np.arange(10, dtype=float)
    # Send the array to rank 1
    comm.Send(data, dest=1)
elif rank == 1:
    # If we're in rank 1, create an array to receive the data
    data = np.empty(10, dtype=float)
    # data will initially contain junk values
    print('data before: ', data)
    # Receive the data from rank 0
    comm.Recv(data, source=0)
    print(rank, data)
```

This code can be found in the file [`04_mpi_scripts/numpy_send_recv.py`](04_mpi_scripts/numpy_send_recv.py) and should be run with two processes.

## Exercise: Sum of Powers of a Array

The function `random_float_array` in the file [`sum_of_powers.py`](04_mpi_scripts/sum_of_powers.py) generates a random array of integers. You should generate a random array of 100 floats with a minimum of zero and maximum of 10 using the function call `random_float_array(0, 10, 100)` on rank 0. Then send this array to all other ranks. Each rank, including rank 0, should calculate the value:

$$
\sum_{i=0}^{n-1} x_i^{r+1}
$$

where $x_i$ is the $i$ th element of the array and $r$ is the rank of the process. Each rank should then send the result back to rank 0. Rank 0 should assemble the results into a list `results` whose $i$ th element is the sum of the powers of the array elements calculated by rank $i$. Finally, rank 0 should print the list of results. This code should be able to be run with any number of ranks. For example, if the code is run on 2 ranks, you might receive the result:

```
[5079.372714885385, 34822.40206813637]
```

while on four ranks you might get

```
[5022.381368813411, 33480.47733076967, 251675.5452958118, 2018665.9149521096]
```

Note that you will get slightly different results as your array will contain different random numbers. The first entry is the sum of the array and is generated on rank 0, the next value is the sum of the square of the array and is calculated on rank 1, the next value is the sum of the cube of the array and is calculated on rank 2, and so on.

There is a sample solution in the file [`sample_solutions/sum_of_powers_solution.py`](sample_solutions/sum_of_powers.py).

## Collective Communication

In the last exercise, we send data from one rank to all other ranks. This sort of communication is a common thing you might want to do in parallel programs. MPI allows for collective communication which provides a convenient and efficient way to do this. The most common collective communication functions are `bcast`, `scatter`, `gather`, and `reduce`. These functions are called by all ranks in the communicator and communicates with all other ranks in the communicator.

### Broadcast

The `bcast` function sends a single object from one rank to all other ranks in the communicator. The syntax is:

```python
import mpi4py.MPI as MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # If the rank is 0, set the data to be broadcasted
    data = ['apples', 'bananas', 'cherries', 'dates']
else:
    # If the rank is not 0, we still need the variable to exist
    # Set it to None for now
    data = None

# Broadcast the data from rank 0 to all other ranks
data = comm.bcast(data, root=0)

# Each rank now has a copy of the data
print(f'Rank {rank} has data: {data}')
```

The code above can be found in the file [`04_mpi_scripts/broadcast.py`](04_mpi_scripts/broadcast.py) and should be run with 4 processes. This code is more compact and efficient that the equivalent code using `send` and `recv`.

### Scatter

The `scatter` function sends an array from one rank to all other ranks in the communicator. The array is divided into equal-sized chunks and each chunk is sent to a different rank. The syntax is:

```python